In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import torch 
from tqdm import tqdm
from darts import TimeSeries
from darts.models import TFTModel

In [ ]:
def check_data_folder(folder):
    return os.path.exists(folder) and os.path.isdir(folder)

def generate_date_range(start_date, end_date):
    """
    Generate a list of dates from start_date to end_date.
    """
    return pd.date_range(start=start_date, end=end_date, freq='D').strftime('%Y%m%d').tolist()

def load_data(file_path):
    """
    Load data from a NetCDF file.
    """
    if os.path.exists(file_path):
        return xr.open_dataset(file_path)
    else:
        raise FileNotFoundError(f'File not found: {file_path}')


# main program
## check data folder
data_folder = f'nc4'
if check_data_folder(data_folder):
    print(f'Data folder found: {data_folder}')
else:
    raise FileNotFoundError(f'Data folder not found: {data_folder}')

## load data
start_date = '2024-01-01'
end_date = '2024-01-14'
date_list = generate_date_range(start_date, end_date)

## get location and shape
path = os.path.join(data_folder, f'M2T1NXFLX.5.12.4%3AMERRA2_400.tavg1_2d_flx_Nx.{date_list[0]}.nc4.dap.nc4')
nc4_data = load_data(path)
lat = nc4_data['lat'].values
lon = nc4_data['lon'].values
shape = nc4_data['TLML'].shape
total_locations = shape[1] * shape[2]

## combine data
# 預先讀取第一個檔案以取得 shape 資訊
sample_path = os.path.join(data_folder, f'M2T1NXFLX.5.12.4%3AMERRA2_400.tavg1_2d_flx_Nx.{date_list[0]}.nc4.dap.nc4')
sample_data = load_data(sample_path)
shape_per_file = sample_data['TLML'].shape   # e.g. (24, 361, 576)
time_per_file = len(sample_data['time'])

# 預先配置 array（假設每天 24 筆資料）
total_samples = len(date_list)
combined = np.empty((total_samples * shape_per_file[0], *shape_per_file[1:]), dtype=np.float32)
time_list = np.empty(total_samples * time_per_file, dtype=sample_data['time'].dtype)

# 批次載入並填入預分配的 array
for i, date in enumerate(tqdm(date_list, desc="Combining")):
    path = os.path.join(data_folder, f'M2T1NXFLX.5.12.4%3AMERRA2_400.tavg1_2d_flx_Nx.{date}.nc4.dap.nc4')
    nc4_data = load_data(path)

    start = i * shape_per_file[0]
    end = (i + 1) * shape_per_file[0]

    combined[start:end] = nc4_data['TLML'].values
    time_list[start:end] = nc4_data['time'].values

print(f'Combined data shape: {combined.shape}')

In [ ]:
SEED = 123
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False

# to Rcode y1 cell × time
ntot, nlat, nlon = combined.shape
ncell = nlat * nlon
y1 = combined.reshape(ntot, ncell).T

# to Rcode gg cell × (lon, lat)
lon_grid, lat_grid = np.meshgrid(lon, lat)
gg = np.vstack([lon_grid.ravel(), lat_grid.ravel()]).T

# 抽樣
ncell, ntot = y1.shape
T = 200
m = 1000
pickm = np.random.choice(ncell, size=m, replace=False)
y_sub   = y1[pickm, :]      # (m, ntot)
gg_sub  = gg[pickm, :]      # (m, 2)

# DatetimeIndex
time_index = pd.to_datetime(time_list)

df = pd.DataFrame(
    y_sub.T,
    index=time_index,
    columns=[f"cell_{i}" for i in range(m)]
)

# 經緯度 DataFrame
static_df = pd.DataFrame(
    gg_sub,
    index=df.columns,
    columns=["lon","lat"]
)

# to TimeSeries
series = TimeSeries.from_dataframe(df, static_covariates=static_df)
train  = series[:T]   # T 1-200
true_201 = df.iloc[T].values  # 201期真值，shape=(m,)

In [ ]:
from pytorch_lightning.callbacks import TQDMProgressBar
progress_callback = TQDMProgressBar(refresh_rate = 1)

# TFTModel
model = TFTModel(
    input_chunk_length  = 48,  # 模型每次觀察過去 48 期資料
    output_chunk_length = 1,   # 預測下 1 期
    hidden_size         = 64,
    lstm_layers         = 1,
    num_attention_heads = 4,
    dropout             = 0.1,
    random_state        = SEED,
    add_relative_index = True,
    pl_trainer_kwargs   = {
        "accelerator": "cpu",
        "devices": 1,
        "max_epochs": 30,
        "gradient_clip_val": 0.1,
        "callbacks": [progress_callback],
    }
)

In [ ]:
# train and predict
model.fit(train, verbose=False)
forecast_201 = model.predict(n=1)      # shape=(1,5000)
pred_201     = forecast_201.values()[0]

# MSPE
mask     = ~np.isnan(true_201)
mspe_py  = np.mean((pred_201[mask] - true_201[mask])**2)
rmspe_py = np.sqrt(mspe_py)
print(f"Python TFTModel Next-step MSPE = {mspe_py:.6f} | RMSPE = {rmspe_py:.6f}")